# Model evaluation and re-training with AdaPT on Cifar10 dataset

In this notebook you can evaluate different approximate multipliers on various models based on Cifar10 dataset

Steps:
* Select models to load 
* Select number of threads to use
* Choose approximate multiplier 
* Load model for evaluation
* Load dataset
* Run model calibration for quantization
* Run model evaluation
* Run approximate-aware re-training
* Rerun model evaluation

**Note**:
* This notebook should be run on a X86 machine

* Please make sure you have run the installation steps first

In [1]:
import os
import zipfile
import random
import numpy as np
import torch

import requests
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.datasets import CIFAR10
from tqdm import tqdm
import torch.nn as nn

In [2]:
def get_random_seed():
    return 1221 # 121 and 1221

def set_random_seeds():
    torch.manual_seed(get_random_seed())
    np.random.seed(get_random_seed())
    random.seed(get_random_seed())


## Select models to load 

The weights must be downloaded in state_dicts folder.


In [3]:
from models.SDNs.vgg_sdn import vgg16_sdn_bn
from models.SDNs.wideresnet_sdn import wideresnet_sdn_v1
from models.SDNs.mobilenet_sdn import mobilenet_sdn_v1
import models.SDNs.fault_injection as fie

## Select number of threads to use

For optimal performance set them as the number of your cpu threads (not cpu cores)

In [4]:
threads = 20
torch.set_num_threads(threads)

# maybe better performance
%env OMP_PLACES=cores
%env OMP_PROC_BIND=close
%env OMP_WAIT_POLICY=active

env: OMP_PLACES=cores
env: OMP_PROC_BIND=close
env: OMP_WAIT_POLICY=active


## Choose approximate multiplier 

Two approximate multipliers are already provided

**mul8s_acc** - (header file: mul8s_acc.h)   <--  default

**mul8s_1L2H** - (header file: mul8s_1L2H.h)



In order to use your custom multiplier you need to use the provided tool (LUT_generator) to easily create the C++ header for your multiplier. Then you just place it inside the adapt/cpu-kernels/axx_mults folder. The name of the axx_mult here must match the name of the header file. The same axx_mult is used in all layers. 

Tip: If you want explicitly to set for each layer a different axx_mult you must do it from the model definition using the respective AdaPT_Conv2d class of each layer.

In [5]:
axx_mult = 'mul8s_acc'
# axx_mult = 'mul8s_1L2H'

# axx_mult = 'mul8s_1L2L'

# axx_mult = 'mul8s_1L2N'
# axx_mult = 'mul8s_1L12'

## Load model for evaluation

Jit compilation method loads 'on the fly' the C++ extentions of the approximate multipliers. Then the pytorch model is loaded

In [6]:
# model = vgg16_sdn_bn(pretrained=True, axx_mult = axx_mult)
# model = wideresnet_sdn_v1(pretrained=True, axx_mult = axx_mult)
model = mobilenet_sdn_v1(pretrained=True, axx_mult = axx_mult)

model.eval() # for evaluation

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_mul8s_acc/build.ninja...
Building extension module PyInit_conv2d_mul8s_acc...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_mul8s_acc...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_mul8s_acc, skipping build step...
Loading extension module PyInit_conv2d_mul8s_acc...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_mul8s_acc, skipping build step...
Loading extension module PyInit_conv2d_mul8s_acc...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_mul8s_acc, skip

MobileNet_SDN(
  (init_conv): Sequential(
    (0): AdaPT_Conv2d(
      3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
      (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
    )
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layers): ModuleList(
    (0): BlockWOutput(
      (layers): Sequential(
        (0): AdaPT_Conv2d(
          32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False
          (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
          (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
        )
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU

In [7]:
# Set random seeds
set_random_seeds()

# Print names of immediate layers only
for name, layer in model.named_modules():
    print(name)
    
# layers.0.layers.0.quantizer
# layers.0.layers.0.quantizer_w



init_conv
init_conv.0
init_conv.0.quantizer
init_conv.0.quantizer_w
init_conv.1
init_conv.2
layers
layers.0
layers.0.layers
layers.0.layers.0
layers.0.layers.0.quantizer
layers.0.layers.0.quantizer_w
layers.0.layers.1
layers.0.layers.2
layers.0.layers.3
layers.0.layers.3.quantizer
layers.0.layers.3.quantizer_w
layers.0.layers.4
layers.0.layers.5
layers.0.output
layers.1
layers.1.layers
layers.1.layers.0
layers.1.layers.0.quantizer
layers.1.layers.0.quantizer_w
layers.1.layers.1
layers.1.layers.2
layers.1.layers.3
layers.1.layers.3.quantizer
layers.1.layers.3.quantizer_w
layers.1.layers.4
layers.1.layers.5
layers.1.output
layers.2
layers.2.layers
layers.2.layers.0
layers.2.layers.0.quantizer
layers.2.layers.0.quantizer_w
layers.2.layers.1
layers.2.layers.2
layers.2.layers.3
layers.2.layers.3.quantizer
layers.2.layers.3.quantizer_w
layers.2.layers.4
layers.2.layers.5
layers.2.output
layers.2.output.max_pool
layers.2.output.avg_pool
layers.2.output.linear
layers.2.output.linear.quantizer

## Load dataset


In [8]:
def val_dataloader(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)):

    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean, std),
        ]
    )
    dataset = CIFAR10(root="datasets/cifar10_data", train=False, download=True, transform=transform)
    dataloader = DataLoader(
        dataset,
        batch_size=128,
        num_workers=0,
        drop_last=True,
        pin_memory=False,
    )
    return dataloader

transform = T.Compose(
        [
            T.RandomCrop(32, padding=4),
            T.RandomHorizontalFlip(),
            T.ToTensor(),
            T.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225)),
        ]
    )
dataset = CIFAR10(root="datasets/cifar10_data", train=True, download=True, transform=transform)

evens = list(range(0, len(dataset), 10))
trainset_1 = torch.utils.data.Subset(dataset, evens)

data = val_dataloader()

# data_t is used for calibration purposes and is a subset of train-set
data_t = DataLoader(trainset_1, batch_size=128,
                                            shuffle=False, num_workers=0)


Files already downloaded and verified
Files already downloaded and verified


In [9]:
class AddTrigger(object):
    def __init__(self, square_size=5, square_loc=(26,26)):
        self.square_size = square_size
        self.square_loc = square_loc

    def __call__(self, pil_data):
        square = Image.new('L', (self.square_size, self.square_size), 255)
        pil_data.paste(square, self.square_loc)
        return pil_data

class Cifar10_:
    def __init__(self, batch_size=128, add_trigger=False):
        self.batch_size = batch_size
        self.img_size = 32
        self.num_classes = 10
        self.num_test = 10000
        self.num_train = 50000

        normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        self.augmented = T.Compose([T.RandomHorizontalFlip(), T.RandomCrop(32, padding=4),T.ToTensor(), normalize])

        self.normalized = T.Compose([T.ToTensor(), normalize])

        self.aug_trainset =  CIFAR10(root='datasets/cifar10_data', train=True, download=False, transform=self.augmented)
        self.aug_train_loader = torch.utils.data.DataLoader(self.aug_trainset, batch_size=batch_size, shuffle=False, num_workers=0)

        self.trainset =  CIFAR10(root='datasets/cifar10_data', train=True, download=False, transform=self.normalized)
        self.train_loader = torch.utils.data.DataLoader(self.trainset, batch_size=batch_size, shuffle=False)

        self.testset =  CIFAR10(root='datasets/cifar10_data', train=False, download=False, transform=self.normalized)
        self.test_loader = torch.utils.data.DataLoader(self.testset, batch_size=batch_size, shuffle=False, num_workers=0)

        # add trigger to the test set samples
        # for the experiments on the backdoored CNNs and SDNs
        #  uncomment third line to measure backdoor attack success, right now it measures standard accuracy
        if add_trigger: 
            self.trigger_transform = T.Compose([AddTrigger(), T.ToTensor(), normalize])
            self.trigger_test_set = CIFAR10(root='datasets/cifar10_data', train=False, download=False, transform=self.trigger_transform)
            # self.trigger_test_set.test_labels = [5] * self.num_test
            self.trigger_test_loader = torch.utils.data.DataLoader(self.trigger_test_set, batch_size=batch_size, shuffle=False, num_workers=0)

def load_cifar10(batch_size, add_trigger=False):
    cifar10_data = Cifar10_(batch_size=batch_size, add_trigger=add_trigger)
    return cifar10_data

def get_dataset(batch_size=128, add_trigger=False):
    return load_cifar10(batch_size, add_trigger)

t_dataset = get_dataset()
one_batch_dataset = get_dataset(1, False)

## Run model calibration for quantization

Calibrates the quantization parameters 

Need to re-run it each time the model changes

In [10]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib

def collect_stats(model, data_loader, num_batches):
     """Feed data to the network and collect statistic"""

     # Enable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.disable_quant()
                 module.enable_calib()
             else:
                 module.disable()

     for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
         model(image.cpu())
         if i >= num_batches:
             break

     # Disable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.enable_quant()
                 module.disable_calib()
             else:
                 module.enable()

def compute_amax(model, **kwargs):
 # Load calib result
 for name, module in model.named_modules():
     if isinstance(module, quant_nn.TensorQuantizer):
         if module._calibrator is not None:
             if isinstance(module._calibrator, calib.MaxCalibrator):
                 module.load_calib_amax()
             else:
                 module.load_calib_amax(**kwargs)
         print(F"{name:40}: {module}")
 model.cpu()

# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
    # optional - test different calibration methods
    #amax = compute_amax(model, method="mse")
    #amax = compute_amax(model, method="entropy")
    

100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.81s/it]
W0905 21:44:42.301057 140100394760000 tensor_quantizer.py:173] Disable HistogramCalibrator
W0905 21:44:42.301445 140100394760000 tensor_quantizer.py:173] Disable HistogramCalibrator
W0905 21:44:42.301767 140100394760000 tensor_quantizer.py:173] Disable HistogramCalibrator
W0905 21:44:42.302050 140100394760000 tensor_quantizer.py:173] Disable HistogramCalibrator
W0905 21:44:42.302324 140100394760000 tensor_quantizer.py:173] Disable HistogramCalibrator
W0905 21:44:42.303059 140100394760000 tensor_quantizer.py:173] Disable HistogramCalibrator
W0905 21:44:42.303351 140100394760000 tensor_quantizer.py:173] Disable HistogramCalibrator
W0905 21:44:42.303630 140100394760000 tensor_quantizer.py:173] Disable HistogramCalibrator
W0905 21:44:42.303910 140100394760000 tensor_quantizer.py:173] Disable HistogramCalibrator
W0905 21:44:42.304197 140100394760000 tensor_quantizer.py:173] D

W0905 21:44:42.332457 140100394760000 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0905 21:44:42.332910 140100394760000 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0905 21:44:42.333283 140100394760000 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0905 21:44:42.333626 140100394760000 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0905 21:44:42.334268 140100394760000 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0905 21:44:42.334599 140100394760000 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0905 21:44:42.335180 140100394760000 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0905 21:44:42.335521 140100394760000 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0905 21:44:42.336082 140100394760000 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0905 21:44:42.336542 140100394760000

init_conv.0.quantizer                   : TensorQuantizer(8bit per-tensor amax=2.6387 calibrator=HistogramCalibrator quant)
init_conv.0.quantizer_w                 : TensorQuantizer(8bit per-tensor amax=1.9424 calibrator=HistogramCalibrator quant)
layers.0.layers.0.quantizer             : TensorQuantizer(8bit per-tensor amax=5.4440 calibrator=HistogramCalibrator quant)
layers.0.layers.0.quantizer_w           : TensorQuantizer(8bit per-tensor amax=2.1052 calibrator=HistogramCalibrator quant)
layers.0.layers.3.quantizer             : TensorQuantizer(8bit per-tensor amax=7.0419 calibrator=HistogramCalibrator quant)
layers.0.layers.3.quantizer_w           : TensorQuantizer(8bit per-tensor amax=1.8036 calibrator=HistogramCalibrator quant)
layers.1.layers.0.quantizer             : TensorQuantizer(8bit per-tensor amax=4.5778 calibrator=HistogramCalibrator quant)
layers.1.layers.0.quantizer_w           : TensorQuantizer(8bit per-tensor amax=0.9850 calibrator=HistogramCalibrator quant)
layers.1

In [11]:
# from models.SDNs.wideresnet_sdn import WideResNet_SDN

# torch.save(model.state_dict(), 'wideresnet_state_dict_sdnn.pth')

# models = WideResNet_SDN()
# torch.load('wideresnet_state_dict_sdnn.pth', map_location="cpu")
# # models = models.load_state_dict(torch.load('wideresnet_state_dict_sdnn.pth', map_location="cpu"))

In [12]:
# import timeit
# correct = 0
# total = 0

# model.eval()
# start_time = timeit.default_timer()
# with torch.no_grad():
#     for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
#         images, labels = images.to("cpu"), labels.to("cpu")
#         outputs = model(images)[-1]
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
# print(timeit.default_timer() - start_time)
# print('Accuracy of the network on the 10000 test images: %.4f %%' % (100 * correct / total))

In [13]:
# # Test the loaded model
# # model.eval()
# top1_test, top5_test, preds = fie.sdn_test_uncertainty(model, t_dataset.test_loader, "cpu")
# print("Top-1 accuracy:",top1_test)
# print("Top-5 accuracy:",top5_test)

# # Sample prediction result
# print("Correctly predicted?",bool(preds[0][0]),",Confidence:",preds[0][1],",Uncertainty:",preds[0][2])


In [14]:
# # Test early exit capability of the mend model with zero uncertainty threshold and confidence threshold of 0.8
# # uncertainty_threshold = -10
# # confidence_threshold = 0.6

uncertainty_threshold = 8
confidence_threshold = 0.5

fie.sdn_test_early_exits(model, one_batch_dataset.test_loader, confidence_threshold, uncertainty_threshold, "cpu")

KeyboardInterrupt: 

In [ ]:
# import mul8s_1L2L

In [ ]:
from copy import copy

def introduce_fault(model, percent_of_faults, fault_loc = None, layer_to_attack = None):
    model.eval()
    for name, param in model.named_parameters():
        if name in layer_to_attack: 
        
            print("Attacked layer",name)
            print(param.shape)
            w1 = param.data
            wf1 = torch.flatten(w1)
            no_of_faults = int(percent_of_faults * len(wf1)/100)
            if (no_of_faults > len(wf1)):
                no_of_faults = len(wf1)

            print("Number of weights attacked", no_of_faults)
            if fault_loc is None:
                fault_loc = random.sample(range(0, len(wf1)), no_of_faults)
                fault = [random.uniform(-2, 2) for _ in range(len(fault_loc))]
#                 print("fault location", fault)
            
            for i in range(0, len(fault_loc)):
#                 print(f"Fault values, before {wf1[fault_loc[i]]},   after: {-wf1[fault_loc[i]]}")
#                 wf1[fault_loc[i]] = -wf1[fault_loc[i]]
                wf1[fault_loc[i]] = torch.tensor(fault[i])
            
            wf11 = wf1.reshape(w1.shape)
            param.data = wf11
    
    return model

In [ ]:
# FP = ['layers.0.layers.1.0.weight'] # Example layers in vgg16
# FP = ['layers.0.layers.1.0.weight','layers.0.layers.0.2.weight']# Example layers in wideresnet
FP = ["init_conv.weight"] # Example layers in wideresnet
FR = 30

model = introduce_fault(model, FR, None, FP)

In [ ]:
top1_acc, top5_acc, early_output_counts, non_conf_output_counts, conf_violation_counts, unc_viol_with_fault = \
  fie.sdn_test_early_exits(model, one_batch_dataset.test_loader, confidence_threshold, uncertainty_threshold, "cpu")

print("top1_acc, top5_acc, early_output_counts, non_conf_output_counts, conf_violation_counts, unc_viol_with_fault: ",
     top1_acc, top5_acc, early_output_counts, non_conf_output_counts, conf_violation_counts, unc_viol_with_fault)

In [ ]:
# from pytorch_quantization import nn as quant_nn
# from pytorch_quantization import calib

# def collect_stats(model, data_loader, num_batches):
#      """Feed data to the network and collect statistic"""

#      # Enable calibrators
#      for name, module in model.named_modules():
#          if isinstance(module, quant_nn.TensorQuantizer):
#              if module._calibrator is not None:
#                  module.disable_quant()
#                  module.enable_calib()
#              else:
#                  module.disable()

#      for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
#          model(image.cpu())
#          if i >= num_batches:
#              break

#      # Disable calibrators
#      for name, module in model.named_modules():
#          if isinstance(module, quant_nn.TensorQuantizer):
#              if module._calibrator is not None:
#                  module.enable_quant()
#                  module.disable_calib()
#              else:
#                  module.enable()

# def compute_amax(model, **kwargs):
#  # Load calib result
#  for name, module in model.named_modules():
#      if isinstance(module, quant_nn.TensorQuantizer):
#          if module._calibrator is not None:
#              if isinstance(module._calibrator, calib.MaxCalibrator):
#                  module.load_calib_amax()
#              else:
#                  module.load_calib_amax(**kwargs)
#          print(F"{name:40}: {module}")
#  model.cpu()

# # It is a bit slow since we collect histograms on CPU
# with torch.no_grad():
#     stats = collect_stats(model, data_t, num_batches=2)
#     amax = compute_amax(model, method="percentile", percentile=99.99)
    
#     # optional - test different calibration methods
#     #amax = compute_amax(model, method="mse")
#     #amax = compute_amax(model, method="entropy")
    

In [ ]:
# top1_acc, top5_acc, early_output_counts, non_conf_output_counts, conf_violation_counts, unc_viol_with_fault = \
#   fie.sdn_test_early_exits(model, one_batch_dataset.test_loader, confidence_threshold, uncertainty_threshold, "cpu")

# print("top1_acc, top5_acc, early_output_counts, non_conf_output_counts, conf_violation_counts, unc_viol_with_fault: ",
#      top1_acc, top5_acc, early_output_counts, non_conf_output_counts, conf_violation_counts, unc_viol_with_fault)